## Supervised Fine-Tuning for Sentiment Classification

텍스트 분류는 자연어처리의 대표적인 태스크로 주어진 텍스트를 여러개 카테고리로 분류하는 기술입니다.  
대표적인 예로는 감성 분석(Sentiment Analysis)을 들 수 있습니다.  
본 예제에서는 사전학습된 DistilBERT를 활용해서 감성 분석을 파인튜닝하는 과정에 대해 살펴보겠습니다.

### 0. Setup

In [ ]:
# !pip install --user datasets

In [ ]:
# MLP Suwon 설정 필요 
import os

os.environ['REQUESTS_CA_BUNDLE'] = '/etc/ssl/certs/ca-certificates.crt'

proxies = {
'http': '75.17.107.42:8080',
'https': '75.17.107.42:8080'
}

In [ ]:
import ssl

if hasattr(ssl, '_create_unverified_context'):
    ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
!nvidia-smi

### 1. DataSets

허깅페이스 허브에서 제공하는 "emotion" 데이터셋을 파인튜닝에 사용하도록 하겠습니다.

Emotion 데이터셋은 Train Set 16,000개, Validation Set 2,000개, Test Set 2,000개로 구성되어 있습니다.

In [ ]:
from datasets import load_dataset

# 다음 코드를 완성하세요!! ("emotion" 데이터셋 로드)
emotions = load_dataset("emotion")
emotions

In [ ]:
train_ds = emotions["train"]
train_ds

Emotion 데이터셋의 레이블은 총 6개 카데고리로 구성되어 있습니다 (Sadness, Joy, Love, Anger, Fear, Surprise)

In [ ]:
print(train_ds.features)

In [ ]:
print(train_ds[:5])

### 2. DataFrame

데이터셋의 포맷과 카테고리별 분포 등에 대해 살펴 보겠습니다.  

In [ ]:
import pandas as pd

emotions.set_format(type="pandas")
df = emotions["train"][:]
df.head()

In [ ]:
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

In [ ]:
import matplotlib.pyplot as plt

df["label_name"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

In [ ]:
df["Words Per Tweet"] = df["text"].str.split().apply(len)
df.boxplot("Words Per Tweet", by="label_name", grid=False,
           showfliers=False, color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

DataFrame 포맷으로부터 데이터셋 출력 포맷 초기화

In [ ]:
emotions.reset_format()

### 3. Tokenization

텍스트를 토큰으로 분할하고 각 토큰을 정수로 매핑합니다.
\[CLS\]와 \[SEP\]는 시퀀스의 시작과 끝을 의미하며, ##IZING와 같이 '##'은 앞 토큰과 공백으로 분리된 것이 아니라, 
앞의 토큰과 연결된 단어였음을 의미합니다.

In [ ]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
# 다음 코드를 완성하세요!! (사전학습 모델에 사용된 Tokenizer 가져오기)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
text = "Tokenizing text is a core task of NLP."
encoded_text = tokenizer(text)
print(encoded_text)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

In [ ]:
print(tokenizer.convert_tokens_to_string(tokens))

In [ ]:
tokenizer.vocab_size

In [ ]:
tokenizer.model_max_length

Fine-Tuning 학습 데이터를 준비합니다.

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
print(tokenize(emotions["train"][:2]))

In [ ]:
# 다음 코드를 완성하세요!! (Dataset.map Method를 이용하여 emotions 데이터셋의 텍스트를 토크화)
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

In [ ]:
print(emotions_encoded["train"].column_names)

### 4. Fine-Tuning

텍스트 분류 학습을 위해 사전학습된 **DistilBERT** 기반으로 **SquenceClassification** 모델을 구성합니다.

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 6
# 다음 코드를 완성하세요!! (사전학습 모델을 기반으로 Sequence Classification 모델 구성)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

model_size = sum(t.numel() for t in model.parameters())
print(f"Model: {model_size/1000**2:.1f}M parameters")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
# 다음 코드를 완성하세요!! (학습 파라미터 설정: num_train_epochs)
training_args = TrainingArguments(output_dir="test-trainer",
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True)

In [ ]:
from transformers import Trainer

# 다음 코드를 완성하세요!! (Trainer 설정: model, args, tokenizer)
trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

In [ ]:
preds_output = trainer.predict(emotions_encoded["validation"])

In [ ]:
preds_output.metrics

In [ ]:
# model.save_pretrained('./test-trainer/best_model')

### 5. Inference

In [ ]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model.eval()

input_text = "I saw a movie today and it was really good."

inputs = tokenizer(input_text, return_tensors='pt')
inputs = inputs.to(device)

with torch.no_grad():
    outputs = model.forward(**inputs)
    logits = outputs.logits

In [ ]:
probabilities = torch.nn.functional.softmax(logits, dim=-1)
predicted_class_idx = torch.argmax(probabilities, dim=-1).item()

print(f"Input text: {input_text}")
print(f"Predicted class index: {predicted_class_idx}")
print(f"Predicted label: {label_int2str(predicted_class_idx)}")
print(f"Class probability: {probabilities}")